In [55]:
# import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
import nltk
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime
import string

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [56]:
# import data

# import glob
# import os

# path = r'/thaisongs' # use your path
# all_files = glob.glob(os.path.join(path , "/*.csv"))

all_files = ['./thaisongs/bodyslam.csv', './thaisongs/nont_tanont.csv', './thaisongs/tilly_birds.csv']

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

song_df = pd.concat(li, axis=0, ignore_index=True)

In [57]:
lyric_in_round_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(lyric_in_round_brackets)))

lyric_in_square_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
print('Number of square brackets: {}'.format(len(lyric_in_square_brackets)))

lyric_in_curly_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\{(.*?)\}',s))), [])
print('Number of curly brackets: {}'.format(len(lyric_in_curly_brackets)))

Number of round brackets: 180
Number of square brackets: 0
Number of curly brackets: 0


In [58]:
# remove round brackets but not text within
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\(|\)', '', s))

lyric_in_round_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(lyric_in_round_brackets)))

Number of round brackets: 0


In [59]:
dot = sum(list(song_df['lyric'].map(lambda s: re.findall('\.',s))), [])
print('Number of dot: {}'.format(len(dot)))

Number of dot: 82


In [60]:
# remove dot 
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\.', '', s))

dot = sum(list(song_df['lyric'].map(lambda s: re.findall('\.',s))), [])
print('Number of dot: {}'.format(len(dot)))

Number of dot: 0


In [61]:
# count number of lines
song_df['lines'] = song_df['lyric'].map(lambda t: len(re.findall(r'\n', t)))

In [62]:
# remove line breaks
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r' \n|\n', '', s))

In [63]:
lyric_in_round_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
print('Number of round brackets: {}'.format(len(lyric_in_round_brackets)))
song_df.head()

Number of round brackets: 0


,song_name,href,lyric,artist,lines
0,อะ ไนท์ (A Night),/music/thailyric/2681,ดึกดื่นคืนใด มองฟ้าไม่เห็นจันทร์ ไม่เจอะเจอกัน...,bodyslam,15
1,ขอบฟ้า,/music/thailyric/2640,ได้แต่เหลียวมองไปสุดฟ้าไกลมันช่างน่าเสียดายที่...,bodyslam,29
2,คนที่ถูกรัก,/music/thailyric/2641,ใจ ฉันเคยถูกทิ้งเพียงลำพัง รักใครมาแล้วตั้งกี่...,bodyslam,25
3,ความเชื่อ,/music/thailyric/2644,มันเกือบจะล้มมันเหนื่อยมันล้าเหมือนแทบขาดใจเดิ...,bodyslam,45
4,ความรักทำให้คนตาบอด,/music/thailyric/2647,รู้ว่ามันไม่สมควร ที่ยังกวนใจเธอ ฉันขอโทษทีรัก...,bodyslam,21


Tokenise

In [64]:
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus.common import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words

In [65]:
# from thainlplib import ThaiWordSegmentLabeller

In [66]:
import nltk
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

[nltk_data] Downloading package words to C:\Users\FACT-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [67]:
def split_word(text):
            
    
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in th_stop and not i in en_stop]
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

In [68]:
song_df['words'] = song_df['lyric'].map(lambda s: split_word(s))

In [69]:
song_df[['lyric','words']].tail()

,lyric,words
125,เจ็บที่ตรงไหนบอกฉันได้ไหมให้ฉันช่วยดูแม้ไม่อาจ...,"[ปวด, ตรงไหน, ไหม, ดู, รู้, แก้, ไขมัน, ปวด, ข..."
126,ฟ้า และท้องทะเลกับลมนั้นมีอยู่แต่ใครจะรู้ จะรู...,"[ฟ้า, ท้องทะเล, ลม, รู้, รู้, ใจ, ไว, ั, ตะวัน..."
127,ทะเลาะเสียงดัง ข้าวของก็พัง อย่างนี้ประจำบางคร...,"[ทะเลาะ, ความดัง, สินค้า, พัง, ประจำ, ช่วงเวลา..."
128,เธอทนอยู่กับเขามานาน อยู่กับเขามานานฉันทนดูเธอ...,"[ทน, ทน, ดู, ทรมาน, ไหว, ยอมให้, เตือน, กี่, ฟ..."
129,เจอมากี่ครั้งเรื่องรักๆ เดี๋ยวจะเล่าให้ฟังชอบค...,"[เจอ, กี่, เรื่อง, รัก, เดี๋ยว, เล่า, ฟัง, ชอบ..."


In [70]:
song_df['n_words'] = song_df['words'].map(len)

EDA

In [71]:
# number of songs
print('number of songs: ', str(len(song_df)))

# number of artists
print('number of artists: ', str(len(song_df['artist'].unique())))

number of songs:  130
number of artists:  3


In [72]:
# distribution songs per artist
song_count_df = song_df.groupby('artist')[['song_name']].count()
fig = px.histogram(song_count_df, x='song_name', title='Songs per artist', labels={'song_name': 'Songs'})
fig.show()

In [73]:
# distribution words per song
fig = px.histogram(song_df, x='n_words', title='Words per song')
fig.show()

In [74]:
# # create dataframe with lists of artists
# song_df['words_str'] = song_df['words'].map(lambda lst: ' '.join(lst))

# # map text to artists
# words_to_artist = {}
# for tp in song_df[['artist', 'words_str']].itertuples(index=False):
#     artist = tp[0]
#     words = tp[1]
#     if words in words_to_artist:
#         words_to_artist[words].append(artist)
#     else:
#         words_to_artist[words] = [artist]

# # insert list of artists to dataframe
# song_df['artists'] = song_df['words_str'].map(words_to_artist)
# song_df['duplicates'] = song_df['artists'].map(len) - 1

# # convert list of artists to set of artists
# song_df['artists'] = song_df['artists'].map(set)
# song_df['n_artists'] = song_df['artists'].map(len)

# # remove duplicate songs
# artist_text_df = song_df.drop_duplicates('words_str')

Feature['artist'].value_counts() engineering

In [75]:
song_df

,song_name,href,lyric,artist,lines,words,n_words
0,อะ ไนท์ (A Night),/music/thailyric/2681,ดึกดื่นคืนใด มองฟ้าไม่เห็นจันทร์ ไม่เจอะเจอกัน...,bodyslam,15,"[ดึกดื่น, คืน, ใด, ฟ้า, เดือน, เจอะ, เจอกัน, ร...",75
1,ขอบฟ้า,/music/thailyric/2640,ได้แต่เหลียวมองไปสุดฟ้าไกลมันช่างน่าเสียดายที่...,bodyslam,29,"[เหลียว, ฟ้า, ช่าง, น่าเสียดาย, ขอบฟ้า, จบ, เส...",74
2,คนที่ถูกรัก,/music/thailyric/2641,ใจ ฉันเคยถูกทิ้งเพียงลำพัง รักใครมาแล้วตั้งกี่...,bodyslam,25,"[ใจ, ทิ้ง, เพียงลำพัง, รัก, กี่, ผิดหวัง, กี่,...",87
3,ความเชื่อ,/music/thailyric/2644,มันเกือบจะล้มมันเหนื่อยมันล้าเหมือนแทบขาดใจเดิ...,bodyslam,45,"[ล้ม, เหนื่อย, ล้า, เหมือน, แทบ, ขาดใจ, เดิน, ...",130
4,ความรักทำให้คนตาบอด,/music/thailyric/2647,รู้ว่ามันไม่สมควร ที่ยังกวนใจเธอ ฉันขอโทษทีรัก...,bodyslam,21,"[รู้, สมควร, ยั่ว, ขอโทษ, รัก, คอย, หวังดี, มี...",51
...,...,...,...,...,...,...,...
125,ให้เธอหายดี (SuperMom),/music/thailyric/18442,เจ็บที่ตรงไหนบอกฉันได้ไหมให้ฉันช่วยดูแม้ไม่อาจ...,tilly_birds,50,"[ปวด, ตรงไหน, ไหม, ดู, รู้, แก้, ไขมัน, ปวด, ข...",54
126,ให้เธอ,/music/thailyric/19409,ฟ้า และท้องทะเลกับลมนั้นมีอยู่แต่ใครจะรู้ จะรู...,tilly_birds,51,"[ฟ้า, ท้องทะเล, ลม, รู้, รู้, ใจ, ไว, ั, ตะวัน...",80
127,อภัย (Broken),/music/thailyric/15735,ทะเลาะเสียงดัง ข้าวของก็พัง อย่างนี้ประจำบางคร...,tilly_birds,31,"[ทะเลาะ, ความดัง, สินค้า, พัง, ประจำ, ช่วงเวลา...",75
128,อย่าอยู่เลย,/music/thailyric/19971,เธอทนอยู่กับเขามานาน อยู่กับเขามานานฉันทนดูเธอ...,tilly_birds,46,"[ทน, ทน, ดู, ทรมาน, ไหว, ยอมให้, เตือน, กี่, ฟ...",133


## Feature engineering

### Number of words

In [76]:
n_artist = 4
random.seed(0)

artist_select = random.choices(song_df['artist'].unique(), k=n_artist)

song_filter_df = song_df.loc[song_df['artist'].isin(artist_select)]
print('Total number of songs: {}'.format(len(song_filter_df)))
song_filter_df.groupby('artist')[['song_name']].count().reset_index().rename(columns={'song':'songs'})

Total number of songs: 130


,artist,song_name
0,bodyslam,66
1,nont_tanont,36
2,tilly_birds,28


In [77]:
fig = px.box(song_filter_df, x='artist', y='n_words', title='Word count per song by artist')
fig.show()

### Repeated words['unique_words_ratio'])song_filter_df.join(

In [78]:
# number of unique stems
song_df['n_unique_words'] = song_df['words'].map(lambda lst: len(set(lst)))

# ratio of unique stems
song_df['unique_words_ratio'] = song_df['n_unique_words'] / song_df['n_words']

# attach column to selected artists
song_filter_df = song_filter_df.join(song_df['unique_words_ratio'])

In [79]:
fig = px.box(song_filter_df, x='artist', y='unique_words_ratio', title='Ratio of unique words to all words')
fig.show()

### Words per line

In [80]:
# calculate number of words per line
song_df['words_per_line'] = song_df['n_words'] / song_df['lines'].astype(float)

song_filter_df = song_filter_df.join(song_df['words_per_line'])

In [81]:
fig = px.box(song_filter_df, x='artist', y='words_per_line', title='Words per line')
fig.show()

## TFIDF

In [102]:
from sklearn.feature_extraction.text import CountVectorizer

In [103]:
# sample_song = [','.join(tkn) for tkn in song_df['words'][0]]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
tran_vect = cvec.fit_transform(song_df['words'][0])

In [84]:
cvec.vocabulary_

{'ดึกดื่น': 10,
 'คืน': 4,
 'ใด': 40,
 'ฟ้า': 14,
 'เดือน': 29,
 'เจอะ': 27,
 'เจอกัน': 26,
 'รู้': 17,
 'หาย': 21,
 'หมอก': 20,
 'ฉาก': 7,
 'แผ่น': 38,
 'บัง': 12,
 'เอาไว้': 37,
 'ความห่างไกล': 3,
 'มองไม่เห็น': 15,
 'ส่องแสง': 19,
 'ฉาย': 8,
 'ลงมา': 18,
 'ห่างหาย': 24,
 'ค่ำ': 6,
 'คืนนี้': 5,
 'เมฆหมอก': 32,
 'เต็มดวง': 30,
 'ห่าง': 23,
 'เหมือนเคย': 35,
 'เจอะกัน': 28,
 'ไหม': 42,
 'ไกลตา': 41,
 'ห่วงใย': 22,
 'เหมือนเดิม': 36,
 'เสมอ': 33,
 'เปลี่ยนไป': 31,
 'รัก': 16,
 'ถึงกัน': 11,
 'เจอ': 25,
 'คงเดิม': 1,
 'กาย': 0,
 'ความรัก': 2,
 'ใจ': 39,
 'เหมือน': 34,
 'พระจันทร์': 13,
 'ดวง': 9}

In [104]:
tran_vect

<75x43 sparse matrix of type '<class 'numpy.int64'>'
	with 75 stored elements in Compressed Sparse Row format>

In [86]:
len(cvec.vocabulary_)

43

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [88]:
tvec2 = TfidfVectorizer(analyzer=lambda x:x.split(','))
t_feat2 = tvec2.fit_transform(song_df['words'][0])

In [89]:
tvec2.idf_

array([4.63758616, 4.63758616, 4.23212105, 4.63758616, 4.63758616,
       3.94443898, 3.94443898, 4.63758616, 4.63758616, 4.63758616,
       4.63758616, 4.23212105, 3.94443898, 4.63758616, 3.94443898,
       4.63758616, 4.23212105, 3.13350876, 4.63758616, 4.63758616,
       4.63758616, 4.23212105, 4.63758616, 4.63758616, 4.63758616,
       4.23212105, 4.63758616, 4.23212105, 4.63758616, 3.94443898,
       4.23212105, 3.72129543, 4.63758616, 3.94443898, 4.23212105,
       4.63758616, 4.63758616, 4.63758616, 4.23212105, 4.23212105,
       4.63758616, 4.63758616, 4.63758616])

In [90]:
t_feat2.shape

(75, 43)

In [91]:
t_feat2.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [92]:
# t_feat2[:,:5].todense()

matrix([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0

In [93]:
t_feat2[:,:5].todense().shape

(75, 5)

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [95]:
def Tfidf(song):
    tvec = TfidfVectorizer()
    t_feat = tvec.fit_transform(song)
    return t_feat.todense()

In [96]:
song_df['tf_idf_vector'] = song_df['words'].map(lambda s: Tfidf(s))
song_df['tf_idf_score'] = song_df['tf_idf_vector'].map(lambda vec: np.sum(vec))

# join values to selected artists
song_filter_df = song_filter_df.join(song_df[['tf_idf_vector', 'tf_idf_score']])

In [97]:
song_filter_df

,song_name,href,lyric,artist,lines,words,n_words,unique_words_ratio,words_per_line,tf_idf_vector,tf_idf_score
0,อะ ไนท์ (A Night),/music/thailyric/2681,ดึกดื่นคืนใด มองฟ้าไม่เห็นจันทร์ ไม่เจอะเจอกัน...,bodyslam,15,"[ดึกดื่น, คืน, ใด, ฟ้า, เดือน, เจอะ, เจอกัน, ร...",75,0.573333,5.000000,[[[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,61.603470
1,ขอบฟ้า,/music/thailyric/2640,ได้แต่เหลียวมองไปสุดฟ้าไกลมันช่างน่าเสียดายที่...,bodyslam,29,"[เหลียว, ฟ้า, ช่าง, น่าเสียดาย, ขอบฟ้า, จบ, เส...",74,0.500000,2.551724,[[[[[0. 0. 0. 0. ...,64.727922
2,คนที่ถูกรัก,/music/thailyric/2641,ใจ ฉันเคยถูกทิ้งเพียงลำพัง รักใครมาแล้วตั้งกี่...,bodyslam,25,"[ใจ, ทิ้ง, เพียงลำพัง, รัก, กี่, ผิดหวัง, กี่,...",87,0.390805,3.480000,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,61.897828
3,ความเชื่อ,/music/thailyric/2644,มันเกือบจะล้มมันเหนื่อยมันล้าเหมือนแทบขาดใจเดิ...,bodyslam,45,"[ล้ม, เหนื่อย, ล้า, เหมือน, แทบ, ขาดใจ, เดิน, ...",130,0.376923,2.888889,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,95.134520
4,ความรักทำให้คนตาบอด,/music/thailyric/2647,รู้ว่ามันไม่สมควร ที่ยังกวนใจเธอ ฉันขอโทษทีรัก...,bodyslam,21,"[รู้, สมควร, ยั่ว, ขอโทษ, รัก, คอย, หวังดี, มี...",51,0.549020,2.428571,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,41.899495
...,...,...,...,...,...,...,...,...,...,...,...
125,ให้เธอหายดี (SuperMom),/music/thailyric/18442,เจ็บที่ตรงไหนบอกฉันได้ไหมให้ฉันช่วยดูแม้ไม่อาจ...,tilly_birds,50,"[ปวด, ตรงไหน, ไหม, ดู, รู้, แก้, ไขมัน, ปวด, ข...",54,0.555556,1.080000,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0....,45.560478
126,ให้เธอ,/music/thailyric/19409,ฟ้า และท้องทะเลกับลมนั้นมีอยู่แต่ใครจะรู้ จะรู...,tilly_birds,51,"[ฟ้า, ท้องทะเล, ลม, รู้, รู้, ใจ, ไว, ั, ตะวัน...",80,0.362500,1.568627,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,61.899495
127,อภัย (Broken),/music/thailyric/15735,ทะเลาะเสียงดัง ข้าวของก็พัง อย่างนี้ประจำบางคร...,tilly_birds,31,"[ทะเลาะ, ความดัง, สินค้า, พัง, ประจำ, ช่วงเวลา...",75,0.480000,2.419355,[[[[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0....,68.392267
128,อย่าอยู่เลย,/music/thailyric/19971,เธอทนอยู่กับเขามานาน อยู่กับเขามานานฉันทนดูเธอ...,tilly_birds,46,"[ทน, ทน, ดู, ทรมาน, ไหว, ยอมให้, เตือน, กี่, ฟ...",133,0.443609,2.891304,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,107.860336


In [98]:
song_df['n_words']

0       75
1       74
2       87
3      130
4       51
      ... 
125     54
126     80
127     75
128    133
129    103
Name: n_words, Length: 130, dtype: int64

In [99]:
song_df['tf_idf_score']

0       61.603470
1       64.727922
2       61.897828
3       95.134520
4       41.899495
          ...    
125     45.560478
126     61.899495
127     68.392267
128    107.860336
129     90.040867
Name: tf_idf_score, Length: 130, dtype: float64

In [100]:
song_df

,song_name,href,lyric,artist,lines,words,n_words,n_unique_words,unique_words_ratio,words_per_line,tf_idf_vector,tf_idf_score
0,อะ ไนท์ (A Night),/music/thailyric/2681,ดึกดื่นคืนใด มองฟ้าไม่เห็นจันทร์ ไม่เจอะเจอกัน...,bodyslam,15,"[ดึกดื่น, คืน, ใด, ฟ้า, เดือน, เจอะ, เจอกัน, ร...",75,43,0.573333,5.000000,[[[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,61.603470
1,ขอบฟ้า,/music/thailyric/2640,ได้แต่เหลียวมองไปสุดฟ้าไกลมันช่างน่าเสียดายที่...,bodyslam,29,"[เหลียว, ฟ้า, ช่าง, น่าเสียดาย, ขอบฟ้า, จบ, เส...",74,37,0.500000,2.551724,[[[[[0. 0. 0. 0. ...,64.727922
2,คนที่ถูกรัก,/music/thailyric/2641,ใจ ฉันเคยถูกทิ้งเพียงลำพัง รักใครมาแล้วตั้งกี่...,bodyslam,25,"[ใจ, ทิ้ง, เพียงลำพัง, รัก, กี่, ผิดหวัง, กี่,...",87,34,0.390805,3.480000,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,61.897828
3,ความเชื่อ,/music/thailyric/2644,มันเกือบจะล้มมันเหนื่อยมันล้าเหมือนแทบขาดใจเดิ...,bodyslam,45,"[ล้ม, เหนื่อย, ล้า, เหมือน, แทบ, ขาดใจ, เดิน, ...",130,49,0.376923,2.888889,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,95.134520
4,ความรักทำให้คนตาบอด,/music/thailyric/2647,รู้ว่ามันไม่สมควร ที่ยังกวนใจเธอ ฉันขอโทษทีรัก...,bodyslam,21,"[รู้, สมควร, ยั่ว, ขอโทษ, รัก, คอย, หวังดี, มี...",51,28,0.549020,2.428571,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,41.899495
...,...,...,...,...,...,...,...,...,...,...,...,...
125,ให้เธอหายดี (SuperMom),/music/thailyric/18442,เจ็บที่ตรงไหนบอกฉันได้ไหมให้ฉันช่วยดูแม้ไม่อาจ...,tilly_birds,50,"[ปวด, ตรงไหน, ไหม, ดู, รู้, แก้, ไขมัน, ปวด, ข...",54,30,0.555556,1.080000,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0....,45.560478
126,ให้เธอ,/music/thailyric/19409,ฟ้า และท้องทะเลกับลมนั้นมีอยู่แต่ใครจะรู้ จะรู...,tilly_birds,51,"[ฟ้า, ท้องทะเล, ลม, รู้, รู้, ใจ, ไว, ั, ตะวัน...",80,29,0.362500,1.568627,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,61.899495
127,อภัย (Broken),/music/thailyric/15735,ทะเลาะเสียงดัง ข้าวของก็พัง อย่างนี้ประจำบางคร...,tilly_birds,31,"[ทะเลาะ, ความดัง, สินค้า, พัง, ประจำ, ช่วงเวลา...",75,36,0.480000,2.419355,[[[[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0....,68.392267
128,อย่าอยู่เลย,/music/thailyric/19971,เธอทนอยู่กับเขามานาน อยู่กับเขามานานฉันทนดูเธอ...,tilly_birds,46,"[ทน, ทน, ดู, ทรมาน, ไหว, ยอมให้, เตือน, กี่, ฟ...",133,59,0.443609,2.891304,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....,107.860336


In [106]:
similarity_matrix = cosine_similarity(vstack(song_df['tf_idf_vector']), 
                                      vstack(song_df['tf_idf_vector']))
artist_names = song_df['artist'].tolist()
fig = go.Figure(data=go.Heatmap(z=np.flipud(similarity_matrix), x=artist_names, y=list(reversed(artist_names)), 
                                colorscale='balance', zmin=0.5, zmax=1.1))
fig.show()

ValueError: blocks[:,0] has incompatible row dimensions. Got blocks[1,0].shape[1] == 38, expected 43.

In [ ]:
# tfidf_vectorizer = TfidfVectorizer(analyzer = 'word')
# #สุ่มช่วงของ 5 เอกสารที่ติดกันมาทดลองใช้งาน
# tfidf_vector= tfidf_vectorizer.fit_transform(docs[637:642])
# tfidf_array = np.array(tfidf_vector.todense())

# #แปลงเป็น DataFrame เพื่อง่ายแก่การอ่าน
# df = pd.DataFrame(tfidf_array,columns=tfidf_vectorizer.get_feature_names())
# df

NameError: name 'docs' is not defined

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
# t_feat = tvec.fit_transform(tokens_list_j)

NameError: name 'tokens_list_j' is not defined